In [ ]:
%pip install langchain langchain_community langchain_chroma

In [ ]:
%pip install -qU langchain-groq

In [1]:
import getpass
import os

In [ ]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = getpass.getpass('LANGCHAIN_API_KEY: ')
os.environ['GROQ_API_KEY'] = getpass.getpass('GROQ_API_KEY: ')

In [3]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [39]:
loader = WebBaseLoader(
    web_paths=("https://llamaimodel.com/requirements/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=('stats-table')
        )
    ),
)
docs = loader.load()

In [41]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [42]:
from langchain_community.embeddings import OllamaEmbeddings

In [43]:
embedding = OllamaEmbeddings(
    model='llama3.1'
)

In [54]:
# vectorstore.delete_collection()
# vectorstore.reset_collection()

In [46]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)

In [47]:
from langchain_groq import ChatGroq

In [48]:
llm = ChatGroq(model="llama3-8b-8192")

In [49]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

D:\py-envs\langchain_rag\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [51]:
rag_chain.invoke("What's the article about?")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'The article is about the specifications and requirements for the Llama 3.1 models, including the 8B, 70B, and 405B models.'

In [52]:
rag_chain.invoke("what are the requirements for the 8b model?")

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'The requirements for the 8B model are: CPU: Modern processor with at least 8 cores, RAM: Minimum of 16 GB recommended, and GPU: NVIDIA RTX 3090 (24 GB) or RTX 4090 (24 GB) for 16-bit mode.'